In [5]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import chardet

In [31]:
# Cargar el archivo train.csv desde la misma carpeta
E1 = pd.read_csv('2023_Accidentalidad.csv', sep=None, engine='python')
E2 = pd.read_csv('2023_LISTADO_UBICACIONES_NP.csv', sep=None, engine='python')
E3 = pd.read_csv('Actuaciones_limpieza_urbana_2023.csv', sep=None, engine='python')
E4 = pd.read_csv('Actuaciones_limpieza_urbana_2024.csv', sep=None, engine='python')
E5 = pd.read_csv('DATOS_ESTACIONES_JULIO_2024.csv', sep=None, engine='python')
E6 = pd.read_csv('DATOS_ESTACIONES_JUNIO_2024.csv', sep=None, engine='python')
E7 = pd.read_csv('intensidad_media_dias_laborables_2023-2024.csv', sep=None, engine='python')
E8 = pd.read_csv('Listado accidentes 2021.csv', sep=None, engine='python')
E9 = pd.read_csv('UbicacionEstacionesPermanentesSentidos.csv', sep=None, engine='python')
E10 = pd.read_csv('uso-de-autobuses.csv', sep=None, engine='python')

result = pd.concat([E1, E2, E3, E4, E5, E6, E7, E8, E9, E10], axis=0)

print(result.head())


  ﻿num_expediente       fecha     hora            localizacion numero  \
0     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
1     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
2     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
3     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
4     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   

   cod_distrito   distrito tipo_accidente estado_meteorológico tipo_vehiculo  \
0           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
1           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
2           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
3           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
4           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   

   ... Sentido Orient. ﻿Línea  Año Tipo de transporte  \
0  ...     NaN     NaN 

In [32]:
result.to_csv('resultado_concatenado.csv', index=False)


In [34]:
# Cargar el archivo train.csv desde la misma carpeta
td = pd.read_csv('resultado_concatenado.csv')

# Ver las primeras filas del archivo
td.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8440\4021125427.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9,10,11,12,14,17,19,20,21,22,23,24,25,26,27,30,64,65,66,80,82,83,84,85,86,87,88,89,93,94,98,100,101,102,104,105,107) have mixed types. Specify dtype option on import or set low_memory=False.
  td = pd.read_csv('resultado_concatenado.csv')


,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,Sentido,Orient.,﻿Línea,Año,Tipo de transporte,Número anual de pasajeros,Expediciones por día laborable,Viajeros por día,Viajeros por expedición,Kilómetros anuales realizados
0,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Descripción estadística de las columnas numéricas
td.describe()

,cod_distrito,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_droga,LOTE,COD_DISTRITO,MUEBLES 010,MUEBLES RECOG. PROGRAMADA,MUEBLES RECOG. EN CALLE,...,heridos graves,victimas mortales,﻿Estación,Sentido,Año,Número anual de pasajeros,Expediciones por día laborable,Viajeros por día,Viajeros por expedición,Kilómetros anuales realizados
count,48827.000000,25945.000000,48821.000000,4.882100e+04,215.0,378.000000,378.000000,378.000000,378.000000,378.000000,...,1908.000000,1908.000000,120.000000,109.000000,84.000000,8.400000e+01,84.000000,84.000000,84.000000,8.400000e+01
mean,9.936142,10.043708,442165.393601,4.474680e+06,1.0,3.523810,11.000000,28.703349,17.924373,72.823780,...,0.015723,0.002621,30.500000,1.467890,2016.000000,4.859460e+05,64.250000,1809.785714,25.226254,2.753712e+05
std,5.698764,4.806775,3519.487633,4.003969e+03,0.0,1.653134,6.063326,26.264525,22.366553,80.086443,...,0.154513,0.051138,17.390715,0.501273,0.821401,3.867346e+05,32.122085,1400.567958,10.110940,2.273131e+05
min,1.000000,1.000000,429081.000000,4.463487e+06,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,2015.000000,5.053200e+04,12.000000,159.000000,9.833333,4.660000e+02
25%,5.000000,7.000000,439957.000000,4.471611e+06,1.0,2.000000,6.000000,9.251000,6.440250,12.580000,...,0.000000,0.000000,15.750000,1.000000,2015.000000,1.557932e+05,39.750000,614.500000,18.464286,9.254025e+04
50%,10.000000,14.000000,441777.000000,4.474699e+06,1.0,4.000000,11.000000,22.265000,12.010000,51.855000,...,0.000000,0.000000,30.500000,1.000000,2016.000000,3.888020e+05,66.500000,1471.000000,23.778261,2.221730e+05
75%,15.000000,14.000000,444085.000000,4.477475e+06,1.0,5.000000,16.000000,41.468250,22.312500,94.537000,...,0.000000,0.000000,45.250000,2.000000,2017.000000,7.582992e+05,94.750000,2760.750000,30.369403,3.535912e+05
max,21.000000,77.000000,455483.000000,4.495568e+06,1.0,6.000000,21.000000,189.880000,236.754000,840.000000,...,4.000000,1.000000,60.000000,2.000000,2017.000000,1.435925e+06,132.000000,6102.000000,54.000000,1.010235e+06


In [36]:
# Recuento de valores nulos
td.isnull().sum()

num_expediente                    35058
fecha                             35058
hora                              33150
localizacion                      33150
numero                            35061
                                  ...  
Número anual de pasajeros         83804
Expediciones por día laborable    83804
Viajeros por día                  83804
Viajeros por expedición           83804
Kilómetros anuales realizados     83804
Length: 113, dtype: int64

In [37]:
# Buscar valores extraños. Para ello, ver los valores distintos en cada feature
columns = td.columns
n_values = [td[a].unique() for a in td.columns]

cuenta = pd.DataFrame()
cuenta['features'] = columns
cuenta['n_values'] = n_values
cuenta

,features,n_values
0,num_expediente,"[2023S000001, 2023S000002, 2023S000003, 2023S0..."
1,fecha,"[01/01/2023, 02/01/2023, 03/01/2023, 04/01/202..."
2,hora,"[1:15:00, 1:50:00, 2:00:00, 1:35:00, 3:00:00, ..."
3,localizacion,"[AVDA. ALFONSO XIII, 33, CALL. ALCALA / CALL. ..."
4,numero,"[33, 638, 34, 0, 48, 16, 11A, 63, 1, 179, 2, 6..."
...,...,...
108,Número anual de pasajeros,"[nan, 178528.0, 114924.0, 158294.0, 137064.0, ..."
109,Expediciones por día laborable,"[nan, 54.0, 21.0, 40.0, 34.0, 75.0, 66.0, 101...."
110,Viajeros por día,"[nan, 602.0, 469.0, 531.0, 578.0, 666.0, 750.0..."
111,Viajeros por expedición,"[nan, 11.1481481481482, 22.3333333333333, 9.83..."


In [38]:
# Ver porcentaje de valores nulos en cada columna
missing_percentage = td.isnull().mean() * 100
print(missing_percentage)

num_expediente                    41.791436
fecha                             41.791436
hora                              39.516975
localizacion                      39.516975
numero                            41.795012
                                    ...    
Número anual de pasajeros         99.899866
Expediciones por día laborable    99.899866
Viajeros por día                  99.899866
Viajeros por expedición           99.899866
Kilómetros anuales realizados     99.899866
Length: 113, dtype: float64


In [41]:
from sklearn.impute import SimpleImputer


# Aquí cargamos los datos de ejemplo (esto lo reemplazarías por tu propio DataFrame)
data_example = {
    'bus_id': [1, 2, 3, 4],
    'route': ['A1', 'B2', 'C3', 'A1'],
    'capacity': [50, 45, 60, None],
    'occupancy': [30, 40, 50, 45],
    'status': ['active', 'inactive', 'active', None]
}

td = pd.DataFrame(data_example)

# Separar las variables categóricas y numéricas
categorical_vars = td.select_dtypes(include=['object']).columns.tolist()
numerical_vars = td.select_dtypes(include=['number']).columns.tolist()

# Crear imputadores
num_imputer = SimpleImputer(strategy='mean')  # Imputador para variables numéricas
cat_imputer = SimpleImputer(strategy='most_frequent')  # Imputador para variables categóricas

# Imputar las variables numéricas
td[numerical_vars] = pd.DataFrame(num_imputer.fit_transform(td[numerical_vars]), columns=numerical_vars)

# Imputar las variables categóricas
td[categorical_vars] = pd.DataFrame(cat_imputer.fit_transform(td[categorical_vars]), columns=categorical_vars)

# Ver el DataFrame después de la imputación
print(td)


   bus_id route   capacity  occupancy    status
0     1.0    A1  50.000000       30.0    active
1     2.0    B2  45.000000       40.0  inactive
2     3.0    C3  60.000000       50.0    active
3     4.0    A1  51.666667       45.0      None
